In [8]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from helpers import *
import time

'''
Calculate the Black-Scholes implied volatility.

Parameters:	
price (float) – the Black-Scholes option price
S (float) – underlying asset price
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
>>> S = 100
>>> K = 100
>>> sigma = .2
>>> r = .01
>>> flag = 'c'
>>> t = .5
>>> price = black_scholes(flag, S, K, t, r, sigma)
>>> iv = implied_volatility(price, S, K, t, r, flag)
'''

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

"\nCalculate the Black-Scholes implied volatility.\n\nParameters:\t\nprice (float) – the Black-Scholes option price\nS (float) – underlying asset price\nK (float) – strike price\nt (float) – time to expiration in years\nr (float) – risk-free interest rate\nflag (str) – ‘c’ or ‘p’ for call or put.\n>>> S = 100\n>>> K = 100\n>>> sigma = .2\n>>> r = .01\n>>> flag = 'c'\n>>> t = .5\n>>> price = black_scholes(flag, S, K, t, r, sigma)\n>>> iv = implied_volatility(price, S, K, t, r, flag)\n"

In [9]:
start_time = time.time()

earnings_lst = []
starting_date = dt.datetime(2018,8,2)
for date in [starting_date + dt.timedelta(i) for i in range(2)]:
    curr_earning = yahoo_earnings(date)
    curr_earning['Earnings Date'] = date
    earnings_lst.append(curr_earning)
earnings = pd.concat(earnings_lst,axis = 0)
tick_lst = earnings.Tickers.tolist()
vols = current_volatility(tick_lst, roll = 22)
earnings.index = earnings.Tickers
earnings_df = earnings.drop_duplicates().join(vols, how='right').drop_duplicates()
earnings_df['Lower Bound'] = earnings_df['close']*(1 - earnings_df['Implied Move Weekly']/100)
earnings_df['Upper Bound'] = earnings_df['close']*(1 + earnings_df['Implied Move Weekly']/100)
print("--- %s seconds ---" % (time.time() - start_time))
earnings_df.index = earnings_df.Tickers
#earnings_df.sort_values('Earnings Date')

#Display Portion
names = earnings_df[((earnings_df['Call Times'] == 'After Market Close') & 
                     (earnings_df['Earnings Date'] == starting_date)) |
                    ((earnings_df['Call Times'] == 'Before Market Open') & 
                     (earnings_df['Earnings Date'] == starting_date + dt.timedelta(1)))].sort_values('Earnings Date')

No Table
No Table
--- 643.8708038330078 seconds ---


In [10]:
names[names['close'] >= 80]

,Call Times,Current 7 Day Implied,EPS,Implied Move Weekly,Tickers,Earnings Date,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,Lower Bound,Upper Bound
Tickers,,,,,,,,,,,,,
EOG,After Market Close,3.85,1.23,3.49,EOG,2018-08-02,0.197742,0.301706,0.128101,125.57,1.606149,121.187607,129.952393
ANET,After Market Close,9.52,1.7,8.80,ANET,2018-08-02,0.366553,0.653361,0.152110,262.60,5.904975,239.491200,285.708800
DATA,After Market Close,10.35,-0.11,9.41,DATA,2018-08-02,0.324635,0.573372,0.152728,106.14,2.107790,96.152226,116.127774
LYB,Before Market Open,4.10,2.92,3.68,LYB,2018-08-03,0.192966,0.315704,0.080055,110.11,1.346729,106.057952,114.162048
CBOE,Before Market Open,3.71,1.03,3.37,CBOE,2018-08-03,0.262711,0.493414,0.063480,96.36,1.607425,93.112668,99.607332


In [ ]:
start_time = time.time()

focus = ['ABC','RCL','CLX']
focus_vol = current_volatility(focus, roll = 22)
focus_names = focus_vol.join(names[['Call Times', 'Implied Move Weekly', 'Earnings Date']])

focus_names['Lower Bound'] = focus_names['close']*(1 - focus_names['Implied Move Weekly']/100)
focus_names['Upper Bound'] = focus_names['close']*(1 + focus_names['Implied Move Weekly']/100)

print("--- %s seconds ---" % (time.time() - start_time))

focus_names

In [ ]:
start_time = time.time()

max_gap = 5
dte_thresh = 5
money_thresh = 0.1
delta_thresh = 0.03
minimum_prem = 0.15
condors_lst = []
putspread_lst = []
callspread_lst = []
bounds_adj = 0.00

for tick in focus_names.index.drop_duplicates().tolist():
    condors, put_spreads, call_spreads = earnings_condor(tick, max_gap, dte_thresh, money_thresh)
    condors['Ticker'] = tick
    put_spreads['Ticker'] = tick
    call_spreads['Ticker'] = tick
    condors = condors[(abs(condors['Delta']) <= delta_thresh) &
                (condors['P Short Strike'] <= focus_names.loc[tick]['Lower Bound']*(1 - bounds_adj)) &
                (condors['C Short Strike'] >= focus_names.loc[tick]['Upper Bound']*(1 + bounds_adj)) &
                (condors['Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    put_spreads = put_spreads[(put_spreads['Short Put Strike'] <= focus_names.loc[tick]['Lower Bound']) &
                              (put_spreads['put Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    call_spreads = call_spreads[(call_spreads['Short call Strike'] >= focus_names.loc[tick]['Upper Bound']) &
                                (call_spreads['call Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    condors_lst.append(condors)
    putspread_lst.append(put_spreads)
    callspread_lst.append(call_spreads)

earnings_pspreads = pd.concat(putspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_cspreads = pd.concat(callspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_condors = pd.concat(condors_lst).sort_values('RiskRewardRatio', ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prem_thresh = 0.3
rr_thresh = 0.3

filtered_condors = earnings_condors[(earnings_condors['Premium'] >= prem_thresh) &
                                     (earnings_condors['RiskRewardRatio'] >= rr_thresh)]

filtered_pspreads = earnings_pspreads[(earnings_pspreads['put Spread Premium'] >= prem_thresh) &
                                     (earnings_pspreads['RiskRewardRatio'] >= rr_thresh)]

filtered_cspreads = earnings_cspreads[(earnings_cspreads['call Spread Premium'] >= prem_thresh) &
                                     (earnings_cspreads['RiskRewardRatio'] >= rr_thresh)]

focus_names

In [ ]:
filtered_condors

In [ ]:
earnings_condors[earnings_condors['Ticker'] == 'ABC']

In [ ]:
# 